<img src="../asset/LOGO_TSP.jpg" height="100">
<img src="../asset/LOGO_MBDA.png" height="100">

## YOLOv5 Implementation on Phytec IMX8+ NPU *(ie. Neural Processing Unit)*
##### Author : *Iwan Ibnoulouafi*

This is our custom made Notebook which encompasses multiple tools allowing to exploit a choosen **YOLOv5** model.

---

#### List of the **available tools** : 
* **Training** a model on a specified dataset *(uses `train.py`)*
* **Validate** a model on a specified validation dataset *(uses `val.py`)*
* **Detection** to run inference of model on a specified input *(uses `detect.py`)*
* **Prune** a model to a given sparsity and run a validation on the pruned model *(uses `val_prune.py`)*
* **Quantize** a model to `uint8`

> Most of them are directly forked from [Ultralytics YOLOv5 repo](https://github.com/ultralytics/yolov5)

# 1. Setup

Install the dependencies (provided by `requirements.txt`) and check PyTorch

In [ ]:
#Commment if already done
#Use pip instead of pip3 if you're running Python 2.X.X or earlier
!pip3 install -r requirements.txt
import torch
import utils

# 2. Detect

`detect.py` runs YOLOv5 inference on a variety of sources and saving results to `runs/detect`. Example inference sources are:

```shell
python detect.py --source 0  # webcam
                          img.jpg  # image
                          vid.mp4  # video
                          screen  # screenshot
                          path/  # directory
                         'path/*.jpg'  # glob
                         'https://youtu.be/LNwODJXcvt4'  # YouTube
                         'rtsp://example.com/media.mp4'  # RTSP, RTMP, HTTP stream
```

#### Flags description : 

* `--weights` : Path to your model

* `--img` : Inference image input size 

* `--conf` : Objects detected with confidence above this value will be displayed

In [4]:
!python3 detect.py --weights yolov5s.pt --img 640 --conf 0.25 --source data/images

zsh:1: command not found: python


# 3. Validate
`val.py` validates a model's accuracy on a given dataset. 

#### Flags description : 

* `--data` : Path to the dataset `.yaml` file.

* `--verbose` : Displays results by class

* All other flags are the same as `detect.py`

In [ ]:
!python3 val.py --weights yolov5s.pt --data coco128.yaml --img 640 --half

# 4. Train
Train a YOLOv5 model on a given dataset, starting from a pretrained `--weights path_to_model`, or from randomly initialized `--weights '' --cfg path_to_model_yaml`.

**Training result** are saved to `runs/train/`.

#### Flags description :

* `--epoch`: number of epochs for the model training

* `--batch`: defines the batch size

* `--device`: device on which to train (ie. `cuda` for gpu, `mps` for Apple Silicon)

* All other flags are the same as `detect.py` & `val.py`

In [ ]:
!python3 train.py --img 640 --batch 16 --epochs 3 --data coco128.yaml --weights yolov5s.pt --cache

# 5. Prune
Prune a pretrained YOLOv5 model to a given sparsity. The pruning is applied using `utils.torch_utils.prune` which implements **unstructured pruning**.

`val_prune.py` first **prune**, **export** then **validate** the pruned model on a given dataset.

#### Flags description :

* `--sparsity`: percentage of the model that will be set to zero

* All other flags are same as `detect.py`, `val.py` & `train.py`

In [ ]:
!python3 val_prune.py --weights yolov5s.pt --data coco128.yaml --img 640 --sparsity 0.3 --device mps

# 6. Quantize
Quantize a given YOLOv5 model to `uint8` for compatibility with NPU. This code is heavily inspired by **Phytec IMX-8+ Software Documentation**.

This code first export the model to **TF saved_model format** and the quantize it in a **TFLite format** using **PTQ (Post-Training Quantization)**

For calibration of the model during the quantization, a **calibration dataset** is required.

In [ ]:
#Use to quantize a pruned model
!python3 export_for_prune.py --weights yolov5s_prune_0.3.pt --include saved_model --device mps

#Use to quantize other models
#!python3 prune.py --weights yolov5s.pt --include saved_model --device mps

In [ ]:
import tensorflow as tf
import os
import numpy as np
import cv2 as cv

#Path to the calibration dataset image's folder
dataset_dir = "../datasets/mycoco100/images/val2017"

def representative_dataset_gen():
    img_file = [os.path.join(dataset_dir, i) for i in os.listdir(dataset_dir) if i.endswith('.jpg')]

    for img_path in img_file:
        img = cv.imread(img_path)
        # Resize image to the input size expected by the model (e.g., 640x640 for YOLOv5)
        img_resized = cv.resize(img, (640, 640))
        # Normalize image to range [0, 1] if required
        img_normalized = img_resized / 255.0
        # Convert to float32 if the model expects it
        img_input = np.expand_dims(img_normalized, axis=0).astype(np.float32)
        # Yield the preprocessed image
        yield [img_input]

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model("yolov5s_prune_0.3_saved_model")
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

converter.target_spec.supported_types = [tf.int8]

#Define the input image type of the quantized model
converter.inference_input_type = tf.float32 

#Define the output image type of the quantized model
converter.inference_output_type = tf.int8

tflite_quant_model = converter.convert()

open('yolov5s_quant.tflite', 'wb').write(tflite_quant_model)

# 7. Export (and Quantize)

Export a given YOLOv5 model from a `.pt` format to a given format *(see `export.py` for available formats)*

#### Flags description :

* `--int8`: Apply `int8` quantization for Tensorflows model.

* `--imgsz`: Correspond to `--img` in the other scripts

* All other flags are the same as `detect.py`, `val.py` & `train.py`

In [ ]:
#Use to export a pruned model
#!python3 export_for_prune.py --weights yolov5s_prune_0.3.pt --include onx

#Use to export other models
!python3 export.py --weights yolov5s.pt --include onnx